# Bllossom-8B 양자화 모델을 이용한 한국어 LLM 튜토리얼

## 01. 활용할 package 설정
 - GPU사용하기: colab에서 런타임 --> 런타임유형변경 --> T4 선택
 - 패키지설치: 아래 pip를 이용해 Transformers, accelerate 설치
 - 런타임재시작: 런타임 --> 세션다시시작  (accelerate설치 시 런타임 다시시작하셔야됩니다!)

In [1]:
!pip install transformers==4.40.0 accelerate
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python #양자화 구동을 위한 Llama C++ 설치
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./' #Bllossom모델 다운로드

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 21.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [2]:
!pip install transformers==4.40.0 accelerate
!pip install llama-cpp-python

In [3]:
from llama_cpp import Llama
from transformers import AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

## 02. 모델준비

In [4]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = Llama(
    model_path='./llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=10000,
    n_gpu_layers=-1        # Number of model layers to offload to GPU
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/10.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ./llama-3-Korean-Bllossom-8B-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 145088
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.f

## 03. 추론

In [21]:
PROMPT = \
'''
Role: "You are a teacher running a college entrance essay interview academy."
Task: "Your main job is to generate example questions from a set of example passages. The number of passages in the set and the number of questions are not fixed. I will show you an example set of passages and example questions. Based on this, you will generate questions from a new set of passages in a similar manner. Your task is to generate only the questions."
'''

instruction = 'Did you understand what I said?'

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True
)

generation_kwargs = {
    "max_tokens":512,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.2,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     538.81 ms
llama_print_timings:      sample time =     111.77 ms /    51 runs   (    2.19 ms per token,   456.29 tokens per second)
llama_print_timings: prompt eval time =     262.38 ms /   111 tokens (    2.36 ms per token,   423.05 tokens per second)
llama_print_timings:        eval time =    1483.32 ms /    50 runs   (   29.67 ms per token,    33.71 tokens per second)
llama_print_timings:       total time =    1905.36 ms /   161 tokens


Yes, I understand that you want me to generate example questions from a set of example passes, based on the style and format of the questions shown to me. I will do my best to come up with new questions for a similar set of passes.


In [27]:
PROMPT = \
'''
Here is a sample set of prompts given as an example. This set consists of three prompts.
[제시문 (가)]
인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가? 칸다울레스 왕이
다스리는 리디아 왕국에 기게스라는 목동이 살았다. 기게스가 양을 치고 있던 어느 날 갑자기
커다란 지진이 일어났다. 지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는
호기심이 생겨 갈라진 동굴 안으로 들어갔다. 그는 동굴 안에서 거인의 시체가 놓여 있는 것을
발견하였다. 시체의 손가락에는 금반지가 끼워져 있었다. 기게스는 그 반지를 빼 들고 밖으로
나왔다. 그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명
인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다. 이제 남들의
시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다.
가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법
반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한
뒤 스스로 왕이 되었다.

[제시문 (나)]
서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고
도덕적인 성향을 보인다. 그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라
판단하고 규범에 맞게 행동한다. 혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이
처신한다. 그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가
발전하면서 대중사회가 등장한다. 이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인
분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다. 대중의
일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진
지도자의 역할은 결정적이다. 권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을
제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다. 대중은 직관과 감정에
따라 권위적 지도자의 말을 무비판적으로 수용한다. 무리 속의 익명적 개인들은 쉽게 흥분하고
변덕을 부리며 열정을 드러낸다. 그러한 감정 에너지는 때로 대중이 난폭하게 폭력을
행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다.


[제시문 (다)]
도시 문명의 발전은 현대 사회의 주요 특징이다. 과학과 기술의 진전을 통해 이루어진 도시
화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다. 그런데 우리
는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다. 개인
은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다.
그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다. 하지만 그는 누가 누군지 알 수
없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다. 이런 까닭에
대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고
비판받기까지 한다.
그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는
비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다. 도시의 익명성은 마을
공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이
아니라 훨씬 더 인간적이고 해방적인 현상이다. 왜냐하면 도시 생활의 익명성 형태는 인간 삶에
필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과
강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다. 따라서 도시인은 이런 익명성 덕택에
과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라
자유롭고 창의적인 생각을 펼칠 수 있게 되었다.


Next, I will show you the example questions. These questions were generated from the sample set of prompts above.


[예시 질문]
제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.


In this way, the example questions reference the content of the sample prompts, summarizing them or asking about topics that can be considered from them. When creating questions, it's important to consider the relationships between the contents of each prompt.
'''

instruction = '''
Please create questions and answers based on the following prompts. You need to generate questions from the [new prompts]. When generating questions, the content should not imply or suggest the answers. Please delete any unnecessary sentences. The format should be as follows:

["Question 1"
"Question 2"
"Question 3"
"Answer 1"
"Answer 2"
'Answer 3"]

Please create two questions and two answers. As I said, when generating questions, the content should not imply or suggest the answers. When creating questions, it's important to consider the relationships between the contents of each prompt. The questions should be in Korean, and concise, consisting of 1-2 sentences.


[new prompts(가)]
이타주의란 자신에게 발생할 손해를 감수하고 타인에게 이득을 주려는 생각 또는 행동을
뜻한다. 이러한 이타주의에 대해서 다양한 논의가 이루어졌다. 이타주의는 거대한 규모의
현대 자본주의 사회에는 더 이상 어울리지 않는 낡은 덕목에 지나지 않는다는 (a)견해가 있다. 이에 따르면 현대 사회에서는 누군가를 위해서가 아니라 자신만을 위해서 행동한 결과 대규모 협력이 이루어진다. 우리가 시장에서 구매해서 사용하고 있는 물건을 보자. 이들
대부분은 살아가면서 한 번도 마주칠 일이 없는 저 어딘가에 있는 누군가가 ‘자신의 이익을
위해서’ 만든 것이다. 이처럼 시장이라는 메커니즘은 우리 눈에 보이지 않는 누군가의 필요도 충족시켜 주고, 자신의 이익만을 추구하면서 행동하더라도 결과적으로는 사회 전체의 복지를 증진시키는 대규모 협력체계를 만들어 낸다. 따라서 이기심이라는 강력한 충동 앞에서 무너지기 쉬운 이타주의에 기초를 둘 것이 아니라, 이기심만으로도 시장이 잘
작동할 수 있게 하는 법적·제도적 장치를 고안하는 것이 더 중요하다. 한편 생물학자들은 우리의 이기심이 유전자에 각인된 것이라고 본다. 이 (b)견해에 따르면 유전자가 살아남기 위한 자기복제의 메커니즘이 모든 생명체가 가진 생족 본능, 즉
이기심의 뿌리다. 하지만 그들은 이기심이 이타주의와 모순된다고 보는 것은 아니며 진화를 통해 이타주의가 형성되었다는 점을 인정한다. 어떤 성향이 진화한다는 것은 그 성향을 가짐으로써 그렇지 않은 경우에 비해, 자손의 수든 물질적 이득의 크기든, 더 큰
보상을 받게 된다는 것을 의미한다. 이타주의는 사회에서 수행해야 할 역할이 있기 때문에 진화 과정 속에서 살아남을 수 있었다.


[new prompts(나)]
최근 미국 캘리포니아 대학의 한 연구팀은 취학 전인 네 살 남녀 어린이 총 74명을 대상으로 실험을 실시했다. 먼저 아이들이 다양한 게임을 통해 총 20개의 교환권을 획득할수 있게 했다. 게임이 끝나 갈 때 쯤 교환권을 선물로 바꿀 수 있다는 것을 알려 준 후, 아파서 게임에 참여하지 못한 친구들에게 교환권을 기부할 수 있다고 말해 주었다. 이후
연구팀은 비밀 공간에 마련된 상자에 어린이들이 각자 알아서 교환권을 넣도록 했다. 그 결과 총 74명의 어린이 중 40명이 적어도 한 개 이상의 교환권을 기부했으며 한 개도
넣지 않은 아이도 34명에 달했다. 이번 연구는 흥미로운 점이 있다. 실험 전에 어린이들에게 부착한 전극으로 미주신경(vagus nerve)을 모니터한 결과, 기부를 많이 한 어린이의 경우 미주신경이 더욱 활성화되는
것으로 나타난 것이다. 연구를 이끈 밀러 박사는 “미주신경은 신체에서 가장 광범위하게 부교감신경을 지배하는데, 심장 박동과 스트레스를 안정적으로 제어한다.”면서 “우리 감정을편안하고 안정적으로 이끌어 건강을 유지하고 원활한 사회적 관계를 맺는 데 도움이 된다.” 고
설명했다.
'''

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True
)

generation_kwargs = {
    "max_tokens":512,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.2,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     538.81 ms
llama_print_timings:      sample time =     633.11 ms /   266 runs   (    2.38 ms per token,   420.15 tokens per second)
llama_print_timings: prompt eval time =     942.99 ms /   611 tokens (    1.54 ms per token,   647.94 tokens per second)
llama_print_timings:        eval time =    9344.81 ms /   265 runs   (   35.26 ms per token,    28.36 tokens per second)
llama_print_timings:       total time =   11253.26 ms /   876 tokens


[질문 1]
이타주의에 대한 다양한 견해가 있다. 이타주의는 어떤 사회적 현상이라고 볼 수 있고, 어떤 문제점들을 가지고 있나요? 

[답변 1]
이타주의는 현대 사회에서 존재하며, 우리 눈에 보이지 않는 다른 사람(특히 자본주의 사회에서 기업 등)의 이익을 위해서 행동하는 것을 의미한다. 이러한 현상은 큰 규모의 협력이 이루어지며, 개인의 이기심 때문에 발생한다고 볼 수 있다. 이는 도시생활과 같은 환경에서는 익명성과 비인가족 관계가 더욱 뚜렷하게 나타나며, 이로 인해 불만족감과 외로움을 느끼게 된다.

[질문 2]
이타주의와 관련된 다양한 관점을 고려할 때, 어떻게 이를 해결할 수 있다고 생각하시나요? 

[답변 2]
이를 해결하기 위해서는 이기심 자체가 아닌 다른 것을 고려해야 한다. 예를 들어, 시장이라는 메커니즘에서 자신에게 이익을 가져다 주는 것이 아니라 타인을 위해서 행동하게 되면 사회 전체의 복지를 증진시키는 대규모 협력체제를 만들 수 있다. 또한, 유전자에 각인된 것이라는 생물학자들의 견해도 있는데, 이타주의는 생존을 위한 자기복제의 메커니즘이며, 이를 이해하는 것이 필요하다. 이러한 해결책들은 이타주의에 대한 새로운 시각과 함께, 사회적 발전과 협력으로 이어질 수 있다.
